# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcf7c01bc40>
├── 'a' --> tensor([[-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288]])
└── 'x' --> <FastTreeValue 0x7fcf7c01ba90>
    └── 'c' --> tensor([[ 1.5529,  1.7956, -0.2025,  0.3461],
                        [-0.0190, -1.0067, -1.9212, -1.4128],
                        [-0.4702, -0.7571, -0.5249,  1.1745]])

In [4]:
t.a

tensor([[-0.6034,  0.6068, -0.7473],
        [-2.2070, -0.5334,  1.1288]])

In [5]:
%timeit t.a

74.1 ns ± 0.904 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcf7c01bc40>
├── 'a' --> tensor([[-0.8039, -0.9966, -0.5889],
│                   [ 0.0655,  0.1640,  0.4001]])
└── 'x' --> <FastTreeValue 0x7fcf7c01ba90>
    └── 'c' --> tensor([[ 1.5529,  1.7956, -0.2025,  0.3461],
                        [-0.0190, -1.0067, -1.9212, -1.4128],
                        [-0.4702, -0.7571, -0.5249,  1.1745]])

In [7]:
%timeit t.a = new_value

82.4 ns ± 0.348 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288]]),
    x: Batch(
           c: tensor([[ 1.5529,  1.7956, -0.2025,  0.3461],
                      [-0.0190, -1.0067, -1.9212, -1.4128],
                      [-0.4702, -0.7571, -0.5249,  1.1745]]),
       ),
)

In [10]:
b.a

tensor([[-0.6034,  0.6068, -0.7473],
        [-2.2070, -0.5334,  1.1288]])

In [11]:
%timeit b.a

68.4 ns ± 0.599 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7557,  0.3648,  0.5205],
               [ 0.1263, -0.6813, -2.0985]]),
    x: Batch(
           c: tensor([[ 1.5529,  1.7956, -0.2025,  0.3461],
                      [-0.0190, -1.0067, -1.9212, -1.4128],
                      [-0.4702, -0.7571, -0.5249,  1.1745]]),
       ),
)

In [13]:
%timeit b.a = new_value

574 ns ± 2.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

990 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 48.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

163 µs ± 988 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

161 µs ± 908 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcf7c08fe50>
├── 'a' --> tensor([[[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]],
│           
│                   [[-0.6034,  0.6068, -0.7473],
│                    [-2.2070, -0.5334,  1.1288]]])
└── 'x' --> <FastTreeValue 0x7fcf7c08f100>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

36.1 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcececd3f70>
├── 'a' --> tensor([[-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288],
│                   [-0.6034,  0.6068, -0.7473],
│                   [-2.2070, -0.5334,  1.1288]])
└── 'x' --> <FastTreeValue 0x7fcececd3130>
    └── 'c' --> tensor([[ 1.5529,  1.7956, -0.2025,  0.3461],
                        [-0.0190, -1.0067, -1.9212, -1.4128],
                 

In [23]:
%timeit t_cat(trees)

35.4 µs ± 520 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

62.6 µs ± 241 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]],
       
               [[-0.6034,  0.6068, -0.7473],
                [-2.2070, -0.5334,  1.1288]]]),
    x: Batch(
           c: tensor([[[ 1.5529,  1.7956, -0.2025,  0.3461],
                       [-0.0190, -1.0067, -1.9212, -1.4128],
                       [-0.4702, -0.7571, -0.5249,  1.1745]],
         

In [26]:
%timeit Batch.stack(batches)

88.7 µs ± 720 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288],
               [-0.6034,  0.6068, -0.7473],
               [-2.2070, -0.5334,  1.1288]]),
    x: Batch(
           c: tensor([[ 1.5529,  1.7956, -0.2025,  0.3461],
                      [-0.0190, -1.0067, -1.9212, -1.4128],
                      [-0.4702, -0.7571, -0.5249,  1.1745],
                      [ 1.5529,  1.7956, -0.2025,  0.3461],
                      [-0.0190, -

In [28]:
%timeit Batch.cat(batches)

162 µs ± 317 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

386 µs ± 8.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
